In [1]:
#| default_exp utils

In [2]:
#| hide
%load_ext autoreload
%autoreload 2
from IPython.core.debugger import set_trace

In [3]:
#| export

import pandas as pd
import numpy as np
import math
import json
import pandas as pd
import re
import itertools
import os
from typing import Callable, List, Optional, Tuple
from pathlib import Path

In [4]:
#| export
def get_game_dataframes(game_path, game_id, event_provider, tracking_provider) -> Tuple:
    "Get all the dataframes based on the game path"
    # events df
    events_df = pd.read_csv(
        game_path / f"{tracking_provider}_{event_provider}_mapping.csv",
    )

    # events reduced df
    events_red_df = pd.read_csv(
        game_path / f"{event_provider}_events_reduced.csv",
    ).rename(columns={"eventId": "event_key"})

    events_df = pd.merge(
        events_df,
        events_red_df[
            ["event_key", "playerKey", "opponentKey", "gameId", "possessionTeamId"]
        ],
        on=["event_key"],
    )
    mask = (
        events_df[["event_name", "frameId", "event_secs"]].shift()
        == events_df[["event_name", "frameId", "event_secs"]]
    ).all(axis=1)
    events_df = events_df.loc[~mask].reset_index(drop=True)

    # tracking df
    tracking_df = pd.read_csv(
        game_path / f"{tracking_provider}_tracking_df.csv", low_memory=False
    ).set_index("frameId")

    # lineup df
    lineup_df = pd.read_csv(
        game_path / "lineup.csv",
    )

    # in and out df
    in_out_df = pd.read_csv(
        game_path / "ins_outs.csv",
    ).rename(columns={"player_optaId": "optaId"})

    # merge lineup_df and in_out_df on id column
    merged = pd.merge(
        lineup_df, in_out_df[["optaId", "position_sportLogic"]], on="optaId", how="left"
    )

    # replace NaN values in lineup_df with values from in_out_df
    merged["position_sportLogic_x"] = merged["position_sportLogic_x"].fillna(
        merged["position_sportLogic_y"]
    )

    # drop the extra value column
    lineup_df = merged.drop("position_sportLogic_y", axis=1).rename(
        columns={"position_sportLogic_x": "position_sportLogic"}
    )

    return events_df, tracking_df, lineup_df

In [5]:
game_path = Path(
    "/home/user/Downloads/Data/games/1059704_2019-08-10_Burnley_Southampton"
)
game_id = 1059704
tracking_provider = "secondSpectrum"
events_df, tracking_df, lineup_df = get_game_dataframes(
    game_path=game_path,
    game_id=game_id,
    event_provider="opta",
    tracking_provider=tracking_provider,
)

In [6]:
#| export
def get_teams(events_df):
    "Get the team id of each team in the game given an events df"
    home_team_id = events_df[events_df["playerKey"].str.startswith("home", na=False)][
        "teamId"
    ].values[0]
    away_team_id = events_df[events_df["playerKey"].str.startswith("away", na=False)][
        "teamId"
    ].values[0]
    return home_team_id, away_team_id

In [7]:
home_team_id, away_team_id = get_teams(events_df)

In [19]:
#| export
def get_goalkeepers(lineup_df):
    "Get goalkeepers jersey for each team"
    return (
        lineup_df[
            (lineup_df["side"] == "home") & (lineup_df["position"] == "GK")
        ].jerseyNumber.values[0],
        lineup_df[
            (lineup_df["side"] == "away") & (lineup_df["position"] == "GK")
        ].jerseyNumber.values[0],
    )

In [18]:
home_gk_jersey, away_gk_jersey = get_goalkeepers(lineup_df)

In [10]:
#| hide
from nbdev import nbdev_export

nbdev_export()